In [56]:
import requests
from bs4 import BeautifulSoup 
import string
import pandas as pd
import csv


In [16]:
completedURL = requests.get("http://www.ufcstats.com/statistics/events/completed?page=all")
fightURL = "http://www.ufcstats.com/event-details/ad23903ef3af7406"
soup = BeautifulSoup(completedURL.text, "html.parser")
table = soup.find("table", attrs={"class" : "b-statistics__table-events"}).find("tbody")
aTags = table.find_all("a", attrs={"class" : "b-link"})
links = []
for aTag in aTags:
    links.append(aTag.get("href"))
del links[1]
del links[0]
fightLinks = []
for link in links:
    soup = BeautifulSoup(requests.get(link).text, "html.parser")
    table = soup.find("tbody", attrs={"class" : "b-fight-details__table-body"})
    rows = table.find_all("tr")
    for row in rows:
        fightLinks.append(row["data-link"])


In [17]:
column = ["Link"]
with open("fightLinks.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(column)
    for link in fightLinks:
        writer.writerow([link])

In [53]:
columns = [
            "Winner", "Loser", "Red", "Blue", "Knockdowns (R)",  "Knockdowns (B)", "Significant Strikes (R)", "Significant Strikes (B)", "Significant Strikes Percentage (R)", "Significant Strikes Percentage (B)", 
           "Total Strikes (R)", "Total Strikes (B)", "Takedowns (R)", "Takedowns (B)",
           "Takedowns Percentage (R)", "Takedowns Percentage (B)",
           "Submission Attempts (R)", "Submission Attempts (B)", "Reversals (R)", "Reversals (B)",
           "Control Time (seconds) (R)", "Control Time (seconds) (B)", 
        ]
fightsDF = pd.DataFrame(columns=columns)
fightersDF = pd.read_csv("fighterStats.csv")
linksDF = pd.read_csv("fightLinks.csv")
linksList = linksDF["Link"].to_list()
for link in linksList:
   soup = BeautifulSoup(requests.get(link).text,"html.parser")
   fighters = soup.find("div", attrs={"class" : "b-fight-details__persons"})
   names = fighters.find_all("a")
   redFighter = names[0].get_text().rstrip()
   blueFighter = names[1].get_text().rstrip()
   result = fighters.find("i").get_text().strip()
   if redFighter in fightersDF["Name"].values and blueFighter in fightersDF["Name"].values:
      rowData = []
      if result == "W":
         rowData = [
            1, 0, redFighter, blueFighter
         ]
      else:
         rowData = [
            0, 1, redFighter, blueFighter
         ]
      stats = soup.find("div", attrs={"class" : "b-fight-details"}).find_all("section")[1].find_all("td")
      del stats[0]
      for stat in stats:
         bothStats = stat.find_all("p")
         redStat = bothStats[0].get_text()
         if "-" in redStat:
            rowData.append(0.0)
         elif "of" in redStat:
            rowData.append(float(redStat.split("of")[0].strip()))
         elif ":" in redStat:
            rowData.append(convertToSec(redStat))
         elif "%" in redStat:
            rowData.append(float(redStat.replace("%","").strip())/100)
         else:
            rowData.append(float(redStat.strip()))
         blueStat = bothStats[1].get_text()
         if "-" in blueStat:
            rowData.append(0.0)
         elif "of" in blueStat:
            rowData.append(float(blueStat.split("of")[0].strip()))
         elif ":" in blueStat:
            rowData.append(convertToSec(blueStat))
         elif "%" in blueStat:
            rowData.append(float(blueStat.replace("%","").strip())/100)
         else:
            rowData.append(float(blueStat.strip()))
      rowDF = pd.DataFrame([rowData], columns=columns)
      fightsDF = pd.concat([fightsDF, rowDF], ignore_index=True)
print(fightsDF)
fightsDF.to_csv("fightStats.csv")

      
      
      

/tmp/ipykernel_24193/26024410.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  fightsDF = pd.concat([fightsDF, rowDF], ignore_index=True)


     Winner Loser                Red               Blue  Knockdowns (R)  \
0         1     0  Alexandre Pantoja        Kai Asakura             0.0   
1         1     0  Shavkat Rakhmonov  Ian Machado Garry             0.0   
2         1     0         Ciryl Gane   Alexander Volkov             0.0   
3         1     0     Bryce Mitchell        Kron Gracie             0.0   
4         0     1      Nate Landwehr         Dooho Choi             0.0   
...     ...   ...                ...                ...             ...   
6902      1     0      Chuck Liddell        Jeff Monson             0.0   
6903      1     0         Tito Ortiz    Wanderlei Silva             1.0   
6904      1     0        Jeremy Horn      Chuck Liddell             0.0   
6905      1     0      Vitor Belfort    Wanderlei Silva             1.0   
6906      1     0      Randy Couture      Vitor Belfort             0.0   

      Knockdowns (B)  Significant Strikes (R)  Significant Strikes (B)  \
0                0.0     

In [40]:
def convertToSec(time):
    nums = time.split(":")
    return float(nums[0].strip())*60+float(nums[1].strip())

In [57]:
columns = [
            "Name", "Wins", "Losses", "Draws", "Stance", "Reach (inches)", 
           "Weight (pounds)", "Height (inches)", "Significant Strikes Landed per Minute", "Significant Striking Accuracy",
           "Significant Strikes Absorbed per Minute", "Significant Strike Defence (the percentage of opponents strikes that did not land)",
           "Average Takedowns Landed per 15 minutes", "Takedown Accuracy", "Takedown Defense (the percentage of opponents TD attempts that did not land)",
           "Average Submissions Attempted per 15 minutes"
        ]
df = pd.DataFrame(columns=columns)
fightersURL = "http://www.ufcstats.com/statistics/fighters?"
urlList = []
for char in string.ascii_lowercase:
    urlList.append(f"{fightersURL}char={char}&page=all")
for url in urlList:
    soup = BeautifulSoup(requests.get(url).text,"html.parser")
    table = soup.find("table", attrs={"class" : "b-statistics__table"}).find("tbody")
    fighterRows = table.find_all("tr")
    for row in fighterRows:
        tds = row.find_all("td")
        aTag = row.find("a")
        if len(tds) >= 9 and tds[5].get_text().strip()!="--" and tds[6].get_text().strip()!="--" and tds[4].get_text().strip()!="--" and tds[3].get_text().strip()!="--":
            aTag = row.find("a")
            rowData = [
            aTag.get_text().strip() + " " + tds[1].find("a").get_text().strip(), 
            tds[7].get_text().strip(), 
            tds[8].get_text().strip(), 
            tds[9].get_text().strip(), 
            tds[6].get_text().strip(), 
            float(tds[5].get_text().strip().replace('"',"")), 
            float(tds[4].get_text().strip().replace("lbs.", "")), 
            convertToInches(tds[3].get_text().strip().replace('"',""))
            ]
            fighter = BeautifulSoup(requests.get(aTag.get("href")).text,"html.parser")
            stats = fighter.find("div", attrs={"class":"b-list__info-box-left clearfix"})
            leftStats = stats.find("div", attrs={"class" : "b-list__info-box-left"}).find("ul").find_all("li")
            rightStats = stats.find("div", attrs={"class" : "b-list__info-box-right"}).find("ul").find_all("li")
            del rightStats[0]
            for stat in leftStats:
                stat.find("i").decompose() 
                rowData.append(float(stat.get_text().strip().replace("%","").replace('"',"")))
            for stat in rightStats:
                stat.find("i").decompose() 
                rowData.append(float(stat.get_text().strip().replace("%","").replace('"',"")))
            row_df = pd.DataFrame([rowData], columns=columns)
            df = pd.concat([df, row_df], ignore_index=True)
divideCols = ["Significant Striking Accuracy", "Significant Strike Defence (the percentage of opponents strikes that did not land)", "Takedown Accuracy" ,"Takedown Defense (the percentage of opponents TD attempts that did not land)"]
df[divideCols] = df[divideCols] / 100
print(df)
df.to_csv("fighterStats.csv")





/tmp/ipykernel_24193/3435816362.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, row_df], ignore_index=True)


                     Name Wins Losses Draws    Stance  Reach (inches)  \
0         Nariman Abbasov   28      4     0  Orthodox            66.0   
1        Hamdy Abdelwahab    5      0     0  Southpaw            72.0   
2      Mansur Abdul-Malik    7      0     0  Orthodox            79.0   
3     Shamil Abdurakhimov   20      8     0  Orthodox            76.0   
4              Daichi Abe    6      2     0  Orthodox            71.0   
...                   ...  ...    ...   ...       ...             ...   
2331   Zhalgas Zhumagulov   14      9     0    Switch            66.0   
2332           Fares Ziam   16      4     0  Orthodox            75.0   
2333          Cat Zingano   10      4     0  Southpaw            68.0   
2334         Allan Zuniga   13      1     0  Orthodox            70.0   
2335       Virgil Zwicker   15      6     1                      74.0   

      Weight (pounds)  Height (inches)  Significant Strikes Landed per Minute  \
0               155.0             68.0    

In [55]:
def convertToInches(string):
    strings = string.split("'")
    return float(strings[0])*12 + float(strings[1]) 